<a href="https://colab.research.google.com/github/guscldns/Focus-Read/blob/main/%ED%8C%8C%EC%9D%B4%ED%94%84%EB%9D%BC%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 라이브러리 설치 및 불러오기

In [ ]:
!pip install google-cloud-vision
!pip install PyMuPDF
!pip install transformers

In [ ]:
from PIL import Image
from google.cloud import vision
import numpy as np
import fitz
import zipfile
import cv2
import io
import os
import torch
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration

#### 파일 저장할 폴더 생성

In [ ]:
# 파일 저장할 폴더 생성
os.mkdir("/content/upload_file")
os.mkdir("/content/image_file")

#### 파일 업로드

In [ ]:
# 코렙이라서 사용, flask에서는 다른 방법 필요
%cd /content/upload_file
from google.colab import files
uploaded = files.upload()

/content/upload_file


Saving 현진건-운수좋은날+B3356-개벽.pdf to 현진건-운수좋은날+B3356-개벽 (1).pdf


In [ ]:
file_names = os.listdir('/content/upload_file')
if '.ipynb_checkpoints' in file_names:
    file_names.remove('.ipynb_checkpoints')
file_names

['현진건-운수좋은날+B3356-개벽 (1).pdf']

## PDF 혹은 image 파일 ocr

##### PDF to IMG

In [ ]:
def pdf_to_png(files):
    path = f"/content/upload_file/{files}"
    doc = fitz.open(path)
    for i, page in enumerate(doc):
        img = page.get_pixmap()
        img.save(f"/content/image_file/{i}.png")

In [ ]:
file_paths = file_names[0]
print(file_paths)
pdf_to_png(file_paths)

현진건-운수좋은날+B3356-개벽 (1).pdf


##### 함수화

In [ ]:
def detect_paragraphs(image_path):
    from google.cloud import vision
    # API키 가져오기
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"/content/drive/MyDrive/api-project-397750607032-5ddc025931cd.json"

    # API 가져오기
    client = vision.ImageAnnotatorClient()

    # 주석을 추가할 이미지 파일 이름
    file_name = os.path.abspath(image_path)

    # 이미지 로드
    with io.open(file_name, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    # 이미지 OCR
    response = client.document_text_detection(image=image)

    # 이미지 OCR 텍스트 전문
    full_text = response.full_text_annotation.text

    # 이미지 OCR 후 결과 (bbox, word 등)
    pages = response.full_text_annotation.pages

    # 텍스트 주석
    # 참고 : https://cloud.google.com/dotnet/docs/reference/Google.Cloud.Vision.V1/latest/Google.Cloud.Vision.V1.TextAnnotation.Types.DetectedBreak.Types.BreakType
    # 참고 : https://googleapis.github.io/googleapis/java/grpc-google-cloud-vision-v1/0.1.5/apidocs/com/google/cloud/vision/v1/TextAnnotation.DetectedBreak.BreakType.html
    breaks = vision.TextAnnotation.DetectedBreak.BreakType

    paragraphs = []
    lines = []

    for page in pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                para = "★문단시작★"
                line = ""
                for word in paragraph.words:
                    for symbol in word.symbols:
                        line += symbol.text
                        # breaks.SPACE : 공백
                        if symbol.property.detected_break.type == breaks.SPACE:
                            line += ' '
                        # breaks.EOL_SURE_SPACE : 줄 바꿈
                        if symbol.property.detected_break.type == breaks.EOL_SURE_SPACE:
                            line += ' '
                            lines.append(line)
                            para += line
                            line = ''
                        # breaks.LINE_BREAK : 단락을 끝내는 줄바꿈
                        if symbol.property.detected_break.type == breaks.LINE_BREAK:
                            lines.append(line)
                            para += line
                            line = ''
                paragraphs.append(para)

    return full_text, paragraphs

##### 실제 데이터 사용

In [ ]:
# 함수 시험
text_full, paragraphs = [], []
for i in os.listdir('/content/image_file'):
    image_path = f"/content/image_file/{i}"
    if image_path[-4:] == '.png':
        text_full2, paragraphes = detect_paragraphs(image_path)
        text_full.append(text_full2)
        paragraphs.append(paragraphes)
print(text_full)
print(paragraphs)

['이 맺히었다. 김첨지의 눈시울도 뜨끈뜨끈하였다.\n이 환자가 그러고도 먹는 데는 물리지 않았다. 사흘 전부터 설렁탕 국물이\n마시고 싶다고 남편을 졸랐다.\n"이런 오라질 년! 조밥도 못 먹는 년이 설렁탕은 또 처먹고 지랄병을 하\n게."\n라고, 야단을 쳐보았건만, 못 사주는 마음이 시원치는 않았다.\n인제 설렁탕을 사줄 수도 있다. 앓는 어미 곁에서 배고파 보채는 개똥이\n(세살먹이)에게 죽을 사줄 수도 있다 - 팔십 전을 손에 쥔 김 첨지의 마음\n은 푼푼하였다.\n그러나 그의 행운은 그걸로 그치지 않았다. 땀과 빗물이 섞여 흐르는 목덜\n미를 기름주머니가 다된 왜목 수건으로 닦으며, 그 학교 문을 돌아 나올 때\n였다. 뒤에서 "인력거!" 하고 부르는 소리가 난다. 자기를 불러 멈춘 사\n람이 그 학교 학생인 줄 김첨지는 한번 보고 짐작할 수 있었다. 그 학생은\n다짜고짜로,\n"남대문 정거장까지 얼마요."\n라고 물었다. 아마도 그 학교 기숙사에 있는 이로 동기방학을 이용하여 귀\n향하려 함이리라. 오늘 가기로 작정은 하였건만 비는 오고, 짐은 있고 해서\n어찌할 줄 모르다가 마침 김첨지를 보고 뛰어나왔음이리라. 그렇지 않으면\n왜 구두를 채 신지 못해서 질질 끌고, 비록 고구라 양복일망정 노박이로 비\n를 맞으며 김첨지를 뒤쫓아 나왔으랴.\n"남대문 정거장까지 말씀입니까."\n하고 김첨지는 잠깐 주저하였다. 그는 이 우중에 우장도 없이 그 먼 곳을\n철벅거리고 가기가 싫었음일까? 처음 것 둘째 것으로 고만 만족하였음일까?\n아니다 결코 아니다. 이상하게도 꼬리를 맞물고 덤비는 이 행운 앞에 조금\n겁이 났음이다. 그리고 집을 나올 제 아내의 부탁이 마음이 켕기었다 - 앞\n집 마마님한테서 부르러 왔을 제 병인은 뼈만 남은 얼굴에 유일의 샘물 같\n은 유달리 크고 움푹한 눈에 애걸하는 빛을 띄우며,\n"오늘은 나가지 말아요. 제발 덕분에 집에 붙어 있어요. 내가 이렇게 아\n픈데・・・・・.."\n라고, 모기 소리같이 중얼거리고 숨을 걸그렁걸그렁하였

## 요약모델

In [ ]:
# # 모델 불러오기
# %cd /content
# !gdown 1T_dCqTY0RXp9qR3Wlk9i3k9x4iMLR490
# #아래 링크에서 폴더 다운로드 or 서버의 '/home/alpaco/hw/KoBART-summarization/kobart_summary' 폴더 다운받기
# #https://drive.google.com/drive/folders/1kncgLWMal5TtnDPPPaswmRkw1k18Jcsa?usp=drive_link

/content
Downloading...
From: https://drive.google.com/uc?id=1T_dCqTY0RXp9qR3Wlk9i3k9x4iMLR490
To: /content/kobart_summary.zip
100% 460M/460M [00:06<00:00, 68.1MB/s]


In [ ]:
# # unzip
# import zipfile
# import os
# os.mkdir('/content/kobart_summary')
# with zipfile.ZipFile('/content/kobart_summary.zip') as f:
#     f.extractall(path ='/content/kobart_summary')

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration, BartForCausalLM

# 모델과 토크나이저 로드
# model = BartForConditionalGeneration.from_pretrained('/content/kobart_summary')
model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/KoBART-summarization/kobart_summary')
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
# 파일 여러개라 생긴 이중 리스트 통합
paragraphs = sum(paragraphs,[])
paragraphs

['★문단시작★이 맺히었다. 김첨지의 눈시울도 뜨끈뜨끈하였다.',
 '★문단시작★이 환자가 그러고도 먹는 데는 물리지 않았다. 사흘 전부터 설렁탕 국물이 마시고 싶다고 남편을 졸랐다.',
 '★문단시작★"이런 오라질 년! 조밥도 못 먹는 년이 설렁탕은 또 처먹고 지랄병을 하 게."',
 '★문단시작★라고, 야단을 쳐보았건만, 못 사주는 마음이 시원치는 않았다.',
 '★문단시작★인제 설렁탕을 사줄 수도 있다. 앓는 어미 곁에서 배고파 보채는 개똥이 (세살먹이)에게 죽을 사줄 수도 있다 - 팔십 전을 손에 쥔 김 첨지의 마음 은 푼푼하였다.',
 '★문단시작★그러나 그의 행운은 그걸로 그치지 않았다. 땀과 빗물이 섞여 흐르는 목덜 미를 기름주머니가 다된 왜목 수건으로 닦으며, 그 학교 문을 돌아 나올 때 였다. 뒤에서 "인력거!" 하고 부르는 소리가 난다. 자기를 불러 멈춘 사 람이 그 학교 학생인 줄 김첨지는 한번 보고 짐작할 수 있었다. 그 학생은 다짜고짜로,',
 '★문단시작★"남대문 정거장까지 얼마요."',
 '★문단시작★라고 물었다. 아마도 그 학교 기숙사에 있는 이로 동기방학을 이용하여 귀 향하려 함이리라. 오늘 가기로 작정은 하였건만 비는 오고, 짐은 있고 해서 어찌할 줄 모르다가 마침 김첨지를 보고 뛰어나왔음이리라. 그렇지 않으면 왜 구두를 채 신지 못해서 질질 끌고, 비록 고구라 양복일망정 노박이로 비 를 맞으며 김첨지를 뒤쫓아 나왔으랴.',
 '★문단시작★"남대문 정거장까지 말씀입니까."',
 '★문단시작★하고 김첨지는 잠깐 주저하였다. 그는 이 우중에 우장도 없이 그 먼 곳을 철벅거리고 가기가 싫었음일까? 처음 것 둘째 것으로 고만 만족하였음일까? 아니다 결코 아니다. 이상하게도 꼬리를 맞물고 덤비는 이 행운 앞에 조금 겁이 났음이다. 그리고 집을 나올 제 아내의 부탁이 마음이 켕기었다 - 앞 집 마마님한테서 부르러 왔을 제 병인은 뼈만 남은 얼굴에 유일의 샘물 같 은 유달리 크고 움푹한 눈에 애걸하는 빛을 띄우며,',
 '★문단시작★"오늘은

In [ ]:
# 문단 리스트를 512 토큰으로 나누기
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

# 입력문장 리스트
paragraphs

# 모델 입력 데이터 생성
input_data = []

# 현재까지의 토큰 수 초기화
current_token_count = 0

for paragraph in paragraphs:
    # '★문단시작★'를 제거하고 문단을 토큰화
    paragraph_tokens = tokenizer(paragraph.replace('★문단시작★', ''), return_tensors='pt', add_special_tokens=True).input_ids

    # 현재 문단을 추가해도 512 토큰을 넘지 않으면 추가
    if current_token_count + len(paragraph_tokens[0]) <= 512:
        if not input_data:
            input_data.append(paragraph_tokens)
            current_token_count += len(paragraph_tokens[0])
        else:
            # 이미 문단이 추가되어 있는 경우 이어서 추가
            input_data[-1] = torch.cat((input_data[-1], paragraph_tokens), dim=-1)
            current_token_count += len(paragraph_tokens[0])
    else:
        # 512 토큰을 넘어가면 새로운 입력으로 시작
        input_data.append(paragraph_tokens)
        current_token_count = len(paragraph_tokens[0])

## 키워드 추출


In [ ]:
!pip install keybert
!pip install kiwipiepy

In [ ]:
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel

In [ ]:
txt= """<그 애> 작자미상 우리는 개천 쪽으로 문이 난 납작한 집들이 게딱지처럼 따닥따닥 붙어있는 동네에서 자 랐다. 그 동네에선 누구나 그렇듯 그 애와 나도 가난했다. 물론 다른 점도 있었다. 내 아 버지는 번번히 월급이 밀리는 시원찮은 회사의 영업사원이었다. 그 애의 아버지는 한쪽 안구에 개 눈을 박아 넣고 지하철에서 구걸을 했다. 내 어머니는 방 한가운데 산처럼 쌓 아 놓은 개구리 인형에 눈을 박았다. 그 애의 어머니는 청계천 골목에서 커피도 팔고 박 카스도 팔고 이따금 곱창집 뒷 방에서 몸도 팔았다. 우리집은 네 가족이 방 두 개짜리 전세금에 쩔쩔맸고, 그 애는 화장실 옆 천막을 치고 아궁이를 걸어 간이 부엌을 만든 하코방에서 살았다. 나는 어린이날 탕수육을 못 먹고 짜장면만 먹는다고 울었고, 그 애는 엄마가 외박하는 밤이면 아버지의 허리띠를 피해서 맨발로 포도를 다다다닥 달렸다. 말하자면 그렇다. 우리집은 가난했고, 그 애는 불행했다. 가난한 동네는 국민학교도 작았다. 우리는 4학년때 처음 한 반이 되었다. 우연히 그 애 집을 지나가다가 길가로 훤히 드러나는 아궁이에다 라면을 끓이는 그 애를 보았다. 그 애가 입은 늘어난 러닝셔츠엔 김치국물이 묻어 있었고 얼굴엔 김치국물 같은 핏자국이 말라붙어 있었다. 눈싸움인지 서로를 노려보다가 내가 먼저 말했다. 너네 부엌 뽑기 만들 기에 최고다. 나는 집에서 국자와 설탕을 훔쳐왔고, 국자바닥을 까맣게 태우면서 우리는 친구가 되었다. 사정이 좀 풀려서 우리집은 서울 반대편으로 이사를 했다. 아버지는 친척이 소개 시켜준 회사에 나갔다. 월급은 밀리지 않았고 어머니는 부업을 그만두었다. 나는 가끔 그 애에게 편지를 썼다. 크리스마스에는 일년동안 쓴 딱딱한 커버의 일기장을 그 애에게 보내기도 했다. 그 애는 얇은 공책을 하나 보냈다. 일기는 몇 장 되지 않았다. decoded_paragraphs=[3월 4일 개학했다. 선생님한테 맞았다. 6월 1일 딸기를 먹었다. 9월 3일 누나가 아파서 아버지가 화냈다. 11 월 4일 생일이다. 그 애는 딸기를 먹으면 일기를 썼다. 딸기를 먹는 것이 일기를 쓸만한 일이었다. 우리는 중학생이 되었다."""

In [ ]:
# 출력 및 시간 test
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model) # 로드까지 26초
keywords = kw_model.extract_keywords(txt, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
keywords # 모델이 로드된 상태라면 7초,글자수 1041

[('가난한', 0.381),
 ('집들이', 0.3747),
 ('우리집은', 0.3392),
 ('가난했다', 0.3392),
 ('동네는', 0.3289),
 ('4학년때', 0.3283),
 ('동네에선', 0.3246),
 ('불행했다', 0.3224),
 ('동네에서', 0.3208),
 ('있었다', 0.3196)]

In [ ]:
### 키워드 명사로만 제공
from kiwipiepy import Kiwi
from transformers import BertModel
from keybert import KeyBERT

# 텍스트 입력
text= """<그 애> 작자미상 우리는 개천 쪽으로 문이 난 납작한 집들이 게딱지처럼 따닥따닥 붙어있는 동네에서 자 랐다. 그 동네에선 누구나 그렇듯 그 애와 나도 가난했다. 물론 다른 점도 있었다. 내 아 버지는 번번히 월급이 밀리는 시원찮은 회사의 영업사원이었다. 그 애의 아버지는 한쪽 안구에 개 눈을 박아 넣고 지하철에서 구걸을 했다. 내 어머니는 방 한가운데 산처럼 쌓 아 놓은 개구리 인형에 눈을 박았다. 그 애의 어머니는 청계천 골목에서 커피도 팔고 박 카스도 팔고 이따금 곱창집 뒷 방에서 몸도 팔았다. 우리집은 네 가족이 방 두 개짜리 전세금에 쩔쩔맸고, 그 애는 화장실 옆 천막을 치고 아궁이를 걸어 간이 부엌을 만든 하코방에서 살았다. 나는 어린이날 탕수육을 못 먹고 짜장면만 먹는다고 울었고, 그 애는 엄마가 외박하는 밤이면 아버지의 허리띠를 피해서 맨발로 포도를 다다다닥 달렸다. 말하자면 그렇다. 우리집은 가난했고, 그 애는 불행했다. 가난한 동네는 국민학교도 작았다. 우리는 4학년때 처음 한 반이 되었다. 우연히 그 애 집을 지나가다가 길가로 훤히 드러나는 아궁이에다 라면을 끓이는 그 애를 보았다. 그 애가 입은 늘어난 러닝셔츠엔 김치국물이 묻어 있었고 얼굴엔 김치국물 같은 핏자국이 말라붙어 있었다. 눈싸움인지 서로를 노려보다가 내가 먼저 말했다. 너네 부엌 뽑기 만들 기에 최고다. 나는 집에서 국자와 설탕을 훔쳐왔고, 국자바닥을 까맣게 태우면서 우리는 친구가 되었다. 사정이 좀 풀려서 우리집은 서울 반대편으로 이사를 했다. 아버지는 친척이 소개 시켜준 회사에 나갔다. 월급은 밀리지 않았고 어머니는 부업을 그만두었다. 나는 가끔 그 애에게 편지를 썼다. 크리스마스에는 일년동안 쓴 딱딱한 커버의 일기장을 그 애에게 보내기도 했다. 그 애는 얇은 공책을 하나 보냈다. 일기는 몇 장 되지 않았다. decoded_paragraphs=[3월 4일 개학했다. 선생님한테 맞았다. 6월 1일 딸기를 먹었다. 9월 3일 누나가 아파서 아버지가 화냈다. 11 월 4일 생일이다. 그 애는 딸기를 먹으면 일기를 썼다. 딸기를 먹는 것이 일기를 쓸만한 일이었다. 우리는 중학생이 되었다."""
# Kiwi 형태소 분석기 초기화
kiwi = Kiwi()
kiwi.analyze(text)

# BERT 모델 로드
model = BertModel.from_pretrained('skt/kobert-base-v1')

# KeyBERT 모델 초기화
kw_model = KeyBERT(model)

# 키워드 추출
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)

# 명사 키워드 추출 함수 정의
def extract_nouns(keyword_list):
    noun_keywords = []
    keywordscore = []
    for keyword, score in keywords:
        # 형태소 분석 수행
        tokens = kiwi.analyze(keyword)
        # print(tokens)
        for tk in tokens[0][0]:
            # print(tk)
            if tk.tag == "NNG":
                if tk.form not in noun_keywords:
                    noun_keywords.append(tk.form)
                    keywordscore.append((tk.form,score))

    return noun_keywords, keywordscore

In [ ]:
# 명사 키워드 추출
noun_keywords, keywordscore = extract_nouns(keywords)
print('noun_keywords:', noun_keywords)
print('keywordscore:', keywordscore)

noun_keywords: ['가난', '집', '동네', '학년', '때', '불행']
keywordscore: [('가난', 0.381), ('집', 0.3747), ('동네', 0.3289), ('학년', 0.3283), ('때', 0.3283), ('불행', 0.3224)]
